In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import Sequential
import tensorflow.keras.layers as nn

from tensorflow import einsum
from einops import rearrange
from einops.layers.tensorflow import Rearrange

import math
from inspect import isfunction

from functools import partial
from tqdm import tqdm

import numpy as np
import os
import cv2
import pathlib
from glob import glob

import time
from tensorflow.python.data.experimental import AUTOTUNE
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import pandas as pd
import skimage
from skimage.metrics import structural_similarity
import time
from skimage import filters, img_as_ubyte,morphology,measure
from scipy.ndimage import label
import copy
import random

In [ ]:
################数据导入###################
degree5_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-200-5.csv").iloc[:,1:]
degree5_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-300-5.csv").iloc[:,1:]
degree5_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-400-5.csv").iloc[:,1:]
degree15_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-200-5.csv").iloc[:,1:]
degree15_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-300-5.csv").iloc[:,1:]
degree15_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-400-5.csv").iloc[:,1:]
degree25_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-200-5.csv").iloc[:,1:]
degree25_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-300-5.csv").iloc[:,1:]
degree25_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-400-5.csv").iloc[:,1:] ###测试

step_max = int(np.array([degree5_200_5.max(),degree5_300_5.max(),degree5_400_5.max(),
            degree15_200_5.max(),degree15_300_5.max(),degree15_400_5.max(),
            degree25_200_5.max(),degree25_300_5.max(),degree25_400_5.max(),]).max())+500
####建立训练集
###lenght=15, 预测1个,随即时间间隔+物理约束
####5,15,25三类，200，300，400三类
X_data = [] 
X_data_increase = []
X_data_factor = [] 
Y_data = []

########5度###############
for i in range (len(degree5_200_5)-15):
    X_data.append(np.transpose([np.array(degree5_200_5.iloc[i:i+15,1])/step_max,np.array(degree5_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[1,0,0]])
    X_data_increase.append(degree5_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_200_5.iloc[i+15,0])

for i in range (len(degree5_300_5)-15):
    X_data.append(np.transpose([np.array(degree5_300_5.iloc[i:i+15,1])/step_max,np.array(degree5_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[0,1,0]])
    X_data_increase.append(degree5_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_300_5.iloc[i+15,0])

for i in range (len(degree5_400_5)-15):
    X_data.append(np.transpose([np.array(degree5_400_5.iloc[i:i+15,1])/step_max,np.array(degree5_400_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[0,0,1]])
    X_data_increase.append(degree5_400_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_400_5.iloc[i+15,0])

########15度###############
for i in range (len(degree15_200_5)-15):
    X_data.append(np.transpose([np.array(degree15_200_5.iloc[i:i+15,1])/step_max,np.array(degree15_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[1,0,0]])
    X_data_increase.append(degree15_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_200_5.iloc[i+15,0])

for i in range (len(degree15_300_5)-15):
    X_data.append(np.transpose([np.array(degree15_300_5.iloc[i:i+15,1])/step_max,np.array(degree15_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[0,1,0]])
    X_data_increase.append(degree15_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_300_5.iloc[i+15,0])

for i in range (len(degree15_400_5)-15):
    X_data.append(np.transpose([np.array(degree15_400_5.iloc[i:i+15,1])/step_max,np.array(degree15_400_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[0,0,1]])
    X_data_increase.append(degree15_400_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_400_5.iloc[i+15,0])

########25度###############
for i in range (len(degree25_200_5)-15):
    X_data.append(np.transpose([np.array(degree25_200_5.iloc[i:i+15,1])/step_max,np.array(degree25_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,0,1],[1,0,0]])
    X_data_increase.append(degree25_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree25_200_5.iloc[i+15,0])

for i in range (len(degree25_300_5)-15):
    X_data.append(np.transpose([np.array(degree25_300_5.iloc[i:i+15,1])/step_max,np.array(degree25_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,0,1],[0,1,0]])
    X_data_increase.append(degree25_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree25_300_5.iloc[i+15,0])

X_data = np.array(X_data)
X_data_increase = np.array(X_data_increase)
Y_data = np.array(Y_data)
X_data_factor = np.array(X_data_factor)

In [ ]:
### 模型########
def scaled_dot_product_attention(Q, K, V,
                                 dropout_rate=0.1,
                                 training=True,
                                 scope="scaled_dot_product_attention"):
    '''
    Q: Packed queries. 3d tensor. [N, T_q, d_k].
    K: Packed keys. 3d tensor. [N, T_k, d_k].
    V: Packed values. 3d tensor. [N, T_k, d_v].
    key_masks: A 2d tensor with shape of [N, key_seqlen]
    causality: If True, applies masking for future blinding
    dropout_rate: A floating point number of [0, 1].
    training: boolean for controlling droput
    scope: Optional scope for `variable_scope`.
    '''
    d_k = Q.get_shape().as_list()[-1]

    # dot product
    outputs = tf.matmul(Q, tf.transpose(K, [0, 2, 1]))  # (N, T_q, T_k)

    # scale
    outputs /= d_k ** 0.5

    # softmax
    outputs = tf.nn.softmax(outputs)

    # dropout
    outputs = tf.keras.layers.Dropout(dropout_rate)(outputs)

    # weighted sum (context vectors)
    outputs = tf.matmul(outputs, V)  # (N, T_q, d_v)

    return outputs
    
def multihead_attention(queries, keys, values,
                        num_heads=8, 
                        dropout_rate=0.1,
                        training=True,
                        scope="multihead_attention"):
    '''
    queries: A 3d tensor with shape of [N, T_q, d_model].
    keys: A 3d tensor with shape of [N, T_k, d_model].
    values: A 3d tensor with shape of [N, T_k, d_model].
    key_masks: A 2d tensor with shape of [N, key_seqlen]
    num_heads: An int. Number of heads.
    dropout_rate: A floating point number.
    training: Boolean. Controller of mechanism for dropout.
    causality: Boolean. If true, units that reference the future are masked.
    scope: Optional scope for `variable_scope`.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    d_model = queries.get_shape().as_list()[-1]
    # Linear projections
    Q = tf.keras.layers.Dense(d_model)(queries) # (N, T_q, d_model) 
    K = tf.keras.layers.Dense(d_model)(keys) # (N, T_k, d_model)
    V = tf.keras.layers.Dense(d_model)(values)# (N, T_v, d_model)
    
    # Split and concat
    Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, d_model/h)
    K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)
    V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_v, d_model/h)

    # Attention
    outputs =  scaled_dot_product_attention(Q_, K_, V_, dropout_rate, training)

    # Restore shape
    outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, d_model)

    # Residual connection
    outputs = tf.keras.layers.add([queries,outputs])

    # Normalize
    outputs = tf.keras.layers.LayerNormalization()(outputs)
 
    return outputs

def ff(inputs, num_units, scope="positionwise_feedforward"):
    '''
    inputs: A 3d tensor with shape of [N, T, C].
    num_units: A list of two integers.
    scope: Optional scope for `variable_scope`.
    Returns:
      A 3d tensor with the same shape and dtype as inputs
    '''
    # Inner layer
    outputs = tf.keras.layers.Dense(num_units[0], activation=tf.nn.relu)(inputs) 

    # Outer layer
    outputs = tf.keras.layers.Dense(num_units[1])(outputs)

    # Residual connection
    outputs = tf.keras.layers.concatenate([outputs, inputs],axis=-1)

    # Normalize
    outputs = tf.keras.layers.LayerNormalization()(outputs)
    
    return outputs

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000 ** (2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000 ** (2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'dtype': self.dtype,
        })
        return config

def Fatigue_transformer():
    History_input = tf.keras.layers.Input(shape=(15,2)) #历史数据，15个点
    Predicted_input_x = tf.keras.layers.Input(shape=(1)) #预测点的x坐标
    Class_curve = tf.keras.layers.Input(shape=(2,3)) #判定曲线类型
    
    encoder_in = tf.keras.layers.Dense(64)(History_input) #(None,15,64)
    layer1 = multihead_attention(encoder_in,encoder_in,encoder_in)
    layer2 = multihead_attention(layer1, layer1, layer1)
    layer3 = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(layer2)
    layer4 = tf.keras.layers.Flatten()(layer3)
    Layer5 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(layer4)
    ####增量预测
    Increase1 = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(Predicted_input_x)
    ####类别
    Onehot1 = tf.keras.layers.Dense(64)(Class_curve)
    Onehot2 = tf.keras.layers.LSTM(64,activation='tanh',return_sequences=False)(Onehot1)
    ####拼接
    Dense1 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(tf.concat([Layer5,Increase1,Onehot2],-1))
    Dense2 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(Dense1)
    LayNorm = tf.keras.layers.LayerNormalization()(Dense2)
    Dense3 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(LayNorm)
    Output = tf.keras.layers.Dense(1,activation="relu")(Dense3) ###输出预测的百分比
 
    #固化模型 
    model=tf.keras.models.Model(inputs=[History_input,Predicted_input_x,Class_curve],outputs=Output)   
    #model.summary()
    return model

In [ ]:
model = Fatigue_transformer()
model.compile( optimizer='adam',loss=tf.keras.losses.MeanAbsoluteError())
History = model.fit(x=[X_data,X_data_increase,X_data_factor],y=Y_data,batch_size=4,epochs=100)
model.save_weights("./Fatigue_transformer")

In [ ]:
#####训练########
model = Fatigue_transformer()
model.load_weights("./Fatigue_transformer")
History=[]
Batch_size=4
optimizer = tf.keras.optimizers.Adam(1e-5)
for epoch in range(0,50,1):
    count=0
    Average_loss=0
    MinEpochs = int(len(X_data)/Batch_size)
    new_list = np.array([i for i in range (len(X_data))])
    np.random.shuffle(new_list)
    for j in range(MinEpochs):
        count+=1
        tem_list = new_list[j*Batch_size:(j+1)*Batch_size]
        train_X_data = X_data[tem_list]
        train_X_data_increase = X_data_increase[tem_list]
        train_X_data_factor = X_data_factor[tem_list]
        train_Y_data = Y_data[tem_list]
        with tf.GradientTape() as tape:
            Predicted_Y = model([train_X_data,train_X_data_increase, train_X_data_factor]) 
            Loss = tf.keras.losses.MAE(train_Y_data,Predicted_Y)
            
            gradients = tape.gradient(Loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))  
            Average_loss=Average_loss + tf.reduce_mean(Loss)
    Average_loss = Average_loss/count
    History.append([epoch, Average_loss])
    tf.print("=>Epoch%4d  loss:%4.6f" %(epoch, Average_loss))   

    model.save_weights("./Fatigue_transformer")

In [ ]:
###############for test#################
model = Fatigue_transformer()
model.load_weights("./Fatigue_transformer")
X_data_test = [] 
X_data_increase_test = []
X_data_factor_test = [] 
Y_data_test = []
for i in range (len(degree25_400_5)-15):
    X_data_test.append(np.transpose([np.array(degree25_400_5.iloc[i:i+15,1])/step_max,np.array(degree25_400_5.iloc[i:i+15,0])]))
    X_data_factor_test.append([[0,0,1],[0,0,1]])
    X_data_increase_test.append(degree25_400_5.iloc[i+15,1]/step_max)
    Y_data_test.append(degree25_400_5.iloc[i+15,0])

X_data_test = np.array(X_data_test) 
X_data_increase_test = np.array(X_data_increase_test)
X_data_factor_test = np.array(X_data_factor_test)
Y_data_test = np.array(Y_data_test)
    
predicted_results=[]
for i in range(len(X_data_test)):
    predicted_results.append(model([X_data_test[i:i+1],X_data_increase_test[i:i+1],X_data_factor_test[i:i+1]]))

In [ ]:
plt.scatter(np.squeeze(Y_data_test),np.squeeze(predicted_results))
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.grid() 
plt.show()

In [ ]:
#####绘制曲线######
model = Fatigue_transformer()
model.load_weights("./Fatigue_transformer")
initial_data_x = np.zeros(len(degree25_400_5))
initial_data_x[0:20] = degree25_400_5.iloc[0:20,0]
initial_data_inc = degree25_400_5.iloc[:,1]/step_max
for i in range(20,len(initial_data_inc)):
    X_data_test = np.transpose([np.array(degree25_400_5.iloc[i-15:i,1])/step_max,np.array(degree25_400_5.iloc[i-15:i,0])])[np.newaxis,...]
    X_data_increase_test = np.array(degree25_400_5.iloc[i,1]/step_max)[np.newaxis,...]
    X_data_factor_test_test = np.array([[0,0,1],[0,0,1]])[np.newaxis,...]
    initial_data_x[i]= model([X_data_test,X_data_increase_test,X_data_factor_test_test]) 

In [ ]:
plt.plot(initial_data_inc*step_max,initial_data_x)
plt.plot(initial_data_inc*step_max,degree25_400_5.iloc[:,0])
plt.show()
Ontest_set = np.concatenate((np.array(initial_data_inc*step_max)[...,np.newaxis],
                             np.array(initial_data_x)[...,np.newaxis],
                             np.array(degree25_400_5.iloc[:,0])[...,np.newaxis]),-1)
Ontest_set = pd.DataFrame(Ontest_set)
Ontest_set.to_excel("D:/AI in NTU/PINN-fatigue/实验结果汇总/transformer_testset.xlsx")

In [ ]:
###############for test#################
model = Fatigue_transformer()
model.load_weights("./Fatigue_transformer")
    
predicted_results=[]
for i in range(len(X_data)):
    predicted_results.append(model([X_data[i:i+1],X_data_increase[i:i+1],X_data_factor[i:i+1]]))
    
ON_train_set = np.concatenate((np.squeeze(Y_data)[...,np.newaxis],np.squeeze(predicted_results)[...,np.newaxis]),-1)
ON_train_set = pd.DataFrame(ON_train_set)
ON_train_set.to_excel("D:/AI in NTU/PINN-fatigue/实验结果汇总/transformer_trainset.xlsx")